# Model Development

In this notebook, we will develop predictive models using the [Vehicles.csv](https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data) dataset. dataset. We will explore various machine learning techniques to create models that can accurately predict used car prices. The process will include building base models and optimizing them through techniques such as regularization and hyperparameter tuning.

By the end of this notebook, we will have established a set of foundational models ready for evaluation, aiming to identify the most effective model for predicting used car prices.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import os, zipfile
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

with zipfile.ZipFile('../data/clean_vehicles.csv.zip', 'r') as zipf:
    zipf.extract('clean_vehicles.csv', path='../data')

data = pd.read_csv('../data/clean_vehicles.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55806 entries, 0 to 55805
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         55806 non-null  int64  
 1   year          55806 non-null  int64  
 2   manufacturer  55806 non-null  object 
 3   model         55805 non-null  object 
 4   condition     55806 non-null  object 
 5   odometer      55806 non-null  float64
 6   cylinders     55806 non-null  object 
 7   fuel          55806 non-null  object 
 8   title_status  55806 non-null  object 
 9   transmission  55806 non-null  object 
 10  drive         55806 non-null  object 
 11  type          55806 non-null  object 
 12  paint_color   55806 non-null  object 
 13  state         55806 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 6.0+ MB


## Random Forest

In [2]:
df = data.copy()
X = df.drop(columns='price')
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numerical_features = ['odometer']
ordinal_features = ['year', 'condition', 'title_status']
categorical_features = ['manufacturer', 'model', 'cylinders', 'fuel',
                        'transmission', 'drive', 'type', 'paint_color', 'state']

numerical_transformer = StandardScaler()
ordinal_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('cat', categorical_transformer, categorical_features)
    ])

rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.99)),
    ('model', RandomForestRegressor(n_jobs=-1, random_state=42))
])

rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_error = mean_squared_error(y_test, rf_predictions)
error_price = np.sqrt(rf_error)
print(error_price)

2207.730847976429


## Sampling Subset

In [3]:
data = data.sample(n=10000, random_state=42)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 15418 to 32859
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         10000 non-null  int64  
 1   year          10000 non-null  int64  
 2   manufacturer  10000 non-null  object 
 3   model         10000 non-null  object 
 4   condition     10000 non-null  object 
 5   odometer      10000 non-null  float64
 6   cylinders     10000 non-null  object 
 7   fuel          10000 non-null  object 
 8   title_status  10000 non-null  object 
 9   transmission  10000 non-null  object 
 10  drive         10000 non-null  object 
 11  type          10000 non-null  object 
 12  paint_color   10000 non-null  object 
 13  state         10000 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 1.1+ MB


## Neural Network

In [4]:
df = data.copy()
X = df.drop(columns='price')
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numerical_features = ['odometer']
ordinal_features = ['year', 'condition', 'title_status']
categorical_features = ['manufacturer', 'model', 'cylinders', 'fuel',
                        'transmission', 'drive', 'type', 'paint_color', 'state']

numerical_transformer = StandardScaler()
ordinal_transformer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)
pca = PCA(n_components=0.99)
X_train_pca = pca.fit_transform(X_train_preprocessed)
X_test_pca = pca.transform(X_test_preprocessed)

neural_net = Sequential([
    Dense(256, input_dim=X_train_pca.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.15),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1)
])

neural_net.compile(optimizer='adam', loss='mse')
neural_net.fit(X_train_pca, y_train, epochs=100, batch_size=32, verbose=0, validation_split=0.2)
neural_pred = neural_net.predict(X_test_pca)
neural_mse = mean_squared_error(y_test, neural_pred)
neural_price_error = np.sqrt(neural_mse)
print('----------')
print(neural_price_error)
print(df['price'].describe())

2024-08-02 01:03:58.097559: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-02 01:03:58.099275: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2


2024-08-02 01:03:58.440138: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-08-02 01:03:58.631093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-08-02 01:03:59.768388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 0s 2ms/step
----------
2954.855992881898
count   10,000.00
mean    12,037.85
std      4,712.47
min      5,795.00
25%      7,995.00
50%     10,995.00
75%     15,495.00
max     22,990.00
Name: price, dtype: float64


2024-08-02 01:04:59.535415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [5]:
try:
    print('Script executed successfully')
except:
    print('FAILED')

Script executed successfully
